##### Bibliotecas utilizadas

In [1]:
import requests 
import pandas as pd
import numpy as np
import time
import random
import csv
import copy

##### Gerando dados

In [ ]:
# Utilizado caso ainda não exista uma base de dados

df_final = pd.DataFrame()
contador = 1 
for i in range (4):
    dados = requests.get('http://localhost:3000/api/ep1').json()
    df = pd.DataFrame(dados)
    df['semana']  = contador
    df_final = pd.concat([df, df_final], axis = 0)
    contador +=1



##### Tratando dados

In [ ]:
# Ajustando data 
df_final['date'] = df_final['date'].map(lambda x: time.ctime(x))

In [9]:
# Ordenando dados 
df_final.sort_values('semana', axis = 0, ascending = True, inplace = True)

In [ ]:
# Ajustando valores negativos e NaN e convertendo os números em inteiro
colunas = df_final.columns

for coluna in colunas[2:len(colunas)-1]:
        df_final[coluna] = df_final[coluna].fillna(0) 
        df_final[coluna] = df_final[coluna].apply(lambda x: int(abs(x)))
        

In [85]:
# Ajustando nomes das colunas para a comparação entre elas funcionar. 
df_final.rename(columns={'prod_0':'prod_00', 'prod_1':'prod_01', 'prod_2':'prod_02', \
    'prod_3':'prod_03', 'prod_4':'prod_04', 'prod_5':'prod_05',\
        'prod_6':'prod_06', 'prod_7':'prod_07', 'prod_8':'prod_08', \
        'prod_9':'prod_09'}, inplace = True)

In [229]:
# Gerando valores totais de produtos movimentados por unidade e por kg 

cabecalho = df_final.columns

# São valores que não quero adicionar nos cabeçalhos
valores_invalidos = ['Unnamed: 0', 'id', 'date']

cabecalho_unidades = list(cabecalho[cabecalho <= 'prod_07'])
for i in valores_invalidos: 
    cabecalho_unidades.remove(i)

cabecalho_kg = list(cabecalho[cabecalho > 'prod_07'])
cabecalho_kg.remove('semana')

df_final['total_por_unidade'] = df_final[cabecalho_unidades].sum(axis = 1)
df_final['total_por_kg'] = df_final[cabecalho_kg].sum(axis = 1)

In [230]:
# gerando csv
df_final.to_csv('all_sales.csv', encoding = 'iso-8859-1', sep = ';')

##### Tratando preços

In [238]:
# Quando já existir um csv na base, abrir arquivo.

df_final = pd.read_csv('all_sales.csv', encoding = 'iso-8859-1', sep = ';')

In [239]:
# Gerando preços aleatórios para produtos novos

df_precos = copy.deepcopy(df_final) 

df_precos.drop(list(range(len(df_final))), axis = 0, inplace = True, errors= 'ignore')
df_precos.drop(['id', 'date', 'total_por_unidade', 'total_por_kg', 'Unnamed: 0.2', 'Unnamed: 0.1', 'Unnamed: 0'], axis = 1, errors= 'ignore', inplace = True)

colunas_precos = df_precos.columns

for coluna in colunas_precos: 
    df_precos.loc[0] = [random.randint(10,30) for x in range(df_precos.shape[1])]
df_precos['semana'] = 1

df_precos



,prod_01,prod_02,prod_03,prod_05,prod_08,prod_09,prod_10,prod_15,prod_00,prod_04,prod_06,prod_11,prod_07,semana
0,12,11,22,11,20,17,26,17,16,11,23,28,19,1


In [ ]:
# Gerando preços aleatórios para as demais semanas

cabecalho = df_precos.columns

produtos_unidades = ['prod_00', 'prod_01', 'prod_02', 'prod_03',\
    'prod_04', 'prod_05', 'prod_06', 'prod_07']


posicao_linha = 1
produtos_un = 0
produtos_kg = 0

for valor in cabecalho: 
    if valor in produtos_unidades: 
        print(df_precos[valor][0])
        produtos_un += 1
    else: 
        print(f'maior que 7 {valor}')
        produtos_kg += 1

In [43]:
vendas_2 = pd.read_csv('monthly_sales.csv', sep = ';', encoding='iso-8859-1').copy()

qtde_prod = str(vendas_2.columns).count('prod_')

for i in vendas_2['total']: 

76724
86775


In [29]:
np.exp(1)

2.718281828459045

##### Gerando transações do mês

In [198]:
df_vendas = df_final.copy()

In [199]:
# Reduzindo data
df_vendas['date'] = df_vendas['date'].apply(lambda x: str(x)[4:7] + '-' + str(x)[20:24])


In [200]:
# Eliminando colunas
df_vendas.drop(['Unnamed: 0', 'id','semana'], axis = 1, inplace = True, errors='ignore')


In [201]:
df_vendas = df_vendas.groupby('date').sum()

In [202]:
cabecalho = df_vendas.columns

cabecalho_unidades = list(cabecalho[cabecalho <= 'prod_07'])
cabecalho_kg = list(cabecalho[cabecalho >= 'prod_07'])

df_vendas['total_por_unidade'] = df_vendas[cabecalho_unidades].sum(axis = 1)
df_vendas['total_por_kg'] = df_vendas[cabecalho_kg].sum(axis = 1)


In [96]:
# Gerando csv 
df_vendas.to_csv('monthly_sales.csv', encoding = 'iso-8859-1', sep = ';')

##### Salvando arquivo CSV

In [237]:
df_precos.to_csv('weekly_price.csv', encoding = 'iso-8859-1', sep = ';')
    